In [22]:
from google.colab import drive
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import EarlyStoppingCallback
import torch.nn as nn

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
finance_data = '/content/drive/MyDrive/finance_data.csv'
df = pd.read_csv(finance_data)
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
missing_values = df.isnull().sum()
print(missing_values)

Sentence     0
Sentiment    0
dtype: int64


In [5]:
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

In [6]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize_function(sentences):
    return tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [8]:
X = df['Sentence']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())

In [10]:
train_labels = torch.tensor(y_train.tolist())
test_labels = torch.tensor(y_test.tolist())

In [11]:
class FinancialSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = FinancialSentimentDataset(train_encodings, train_labels)
test_dataset = FinancialSentimentDataset(test_encodings, test_labels)

In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.893500,0.843861
2,0.507300,0.499021
3,0.387800,0.465935
4,0.289500,0.489306
5,0.227600,0.543076
6,0.162100,0.617923


TrainOutput(global_step=882, training_loss=0.4828712843409201, metrics={'train_runtime': 182.6234, 'train_samples_per_second': 255.882, 'train_steps_per_second': 8.049, 'total_flos': 1412037212135256.0, 'train_loss': 0.4828712843409201, 'epoch': 6.0})

In [16]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [17]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

In [18]:
print(f'BERT Accuracy: {accuracy}')
print(report)

BERT Accuracy: 0.804961505560308
              precision    recall  f1-score   support

    negative       0.69      0.21      0.32       172
     neutral       0.81      0.92      0.86       626
    positive       0.82      0.88      0.85       371

    accuracy                           0.80      1169
   macro avg       0.77      0.67      0.68      1169
weighted avg       0.79      0.80      0.78      1169



In [26]:
def categorical_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):

    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)

    ce_loss = -y_true * torch.log(y_pred)

    focal_loss = alpha * torch.pow(1 - y_pred, gamma) * ce_loss

    return torch.mean(torch.sum(focal_loss, axis=-1))

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        num_classes = logits.size(-1)
        labels_one_hot = nn.functional.one_hot(labels, num_classes=num_classes).float()

        loss = categorical_focal_loss(labels_one_hot, torch.softmax(logits, dim=-1))

        return (loss, outputs) if return_outputs else loss

In [27]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.023700,0.043678
2,0.015300,0.051315
3,0.012300,0.060883
4,0.017300,0.051954


TrainOutput(global_step=588, training_loss=0.01825113303097738, metrics={'train_runtime': 90.5792, 'train_samples_per_second': 515.902, 'train_steps_per_second': 16.229, 'total_flos': 941358141423504.0, 'train_loss': 0.01825113303097738, 'epoch': 4.0})

In [29]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [30]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

In [32]:
print(f'BERT with Focal Loss Accuracy: {accuracy}')
print(report)

BERT with Focal Loss Accuracy: 0.7869974337040205
              precision    recall  f1-score   support

    negative       0.46      0.41      0.44       172
     neutral       0.83      0.84      0.83       626
    positive       0.85      0.87      0.86       371

    accuracy                           0.79      1169
   macro avg       0.71      0.71      0.71      1169
weighted avg       0.78      0.79      0.78      1169

